<a href="https://colab.research.google.com/github/zuzanadobes/DRLND-Project-Collaboration-and-Competition/blob/master/Simple_BERT_Tokenizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BERT TOKANIZATION UTILITY

# Summary 
This notebooks demonstrates the use of BERT model for sentimental analysis applied to Twitter texts.
Sentiment analysis is focused around text-mining of the text according for the purpose of classifying "opinions" discovered in the text. For simplest sentiment classifications are positive and negative.

# DATA 
Twitter data is analyzed.

### Introduction to BERT

Google’s BERT produces word representations that are dynamically represented by the words around them. 
BERT allows requires minimal fine-tuning and provide pretrained models that produce good results. BERT is used to extract features, namely word and sentence embedding vectors, from text data. These embeddings are useful for keyword/search expansion, semantic search and information retrieval. An example application is for matching a customer question against a set of already answered questions or searches against  or matching a search query against existing collections of searches.  The representations capture intent and contextual meaning, and more than just matching common keyword or phrases.

The vectors are used as high-quality feature inputs to NLP models which require inputs in the form of vectors, which consit of numbers.  Features like parts of speech need to be transformed into numerical representations. In the past, words have been represented either as uniquely indexed values, or as in the case of Word2Vec neural word embeddings, they are fixed-length feature embeddings, where each word has a fixed representation regardless of the context within which the word appears.


### Understanding BERT
Deep contextualized word representations assigning word vectors to words in context - typically a sentence - instead of 
assigning a vector to each word type. BERT is one of the most popular and successful examples of these embeddings. 
The authors of BERT released several versions of BERT pretrained on massive amounts of data, including a multilingual 
version which supports 104 languages in a single model.

### Summary of steps
Because BERT is a pretrained model that expects input data in a specific format, we will need:

- special tokens to mark the beginning ([CLS]) and separation/end of sentences ([SEP])
- tokens that conforms with the fixed vocabulary used in BERT
- token IDs from BERT’s tokenizer
- mask IDs to indicate which elements in the sequence are tokens and which are padding elements
- segment IDs used to distinguish different sentences
- positional embeddings used to show token position within the sequence

### Representation of word “embeddings”

tokenizer.tokenize("Telekom is the best company to work ")
['tel', '##ek', '##om', 'is', 'the', 'best', 'company', 'to', 'work']

The original word(s) has been split into smaller subwords and characters. The two hash signs are the tokenizer’s way to denote that this subword or character is part of a larger word and preceded by another subword. 
The BERT tokenizer was created with a WordPiece model which creates a fixed-size vocabulary of individual characters, subwords, and words. Since the vocabulary limit size of our BERT tokenizer model is 30,000, the WordPiece model generated a vocabulary that contains all English characters plus the ~30,000 most common words and subwords found in the English language corpus the model is trained on. This vocabulary contains four things:

- Whole words
- Subwords occuring at the front of a word or in isolation 
- Subwords not at the front of a word, which are preceded by ‘##’ 
- Individual characters

### Segmentation
BERT is trained on and expects sentence pairs, using 1s and 0s to distinguish between the two sentences. That is, for each token in “tokenized_text,” we must specify which sentence it belongs to: sentence 0 (a series of 0s) or sentence 1 (a series of 1s). For our purposes, single-sentence inputs only require a series of 1s, so we will create a vector of 1s for each token in our input sentence.

For processing two sentences, each word in the first sentence should be assigned, plus the ‘[SEP]’ token a 0, and all tokens of the second sentence a 1.





In [0]:
import numpy as np
import re
import math
import pandas as pd
import os
from bs4 import BeautifulSoup
import random

In [4]:
import tensorflow as tf
import tensorflow_hub as hub

from tensorflow.keras import layers
!pip install bert
import bert

  Created wheel for bert: filename=bert-2.2.0-cp36-none-any.whl size=3756 sha256=66f96051ab8962f553f0beef49f7b7db1fbf03fa183227a7e257417b7df9fa81
  Stored in directory: /root/.cache/pip/wheels/fe/71/b7/941459453bd38e5d97a8c886361dee19325e9933c9cf88ad46
  Created wheel for erlastic: filename=erlastic-2.0.0-cp36-none-any.whl size=6786 sha256=bbe75b6b22fc594d217aabe120db35bf12bf9b985d90f5e2034d166c5b038eae
  Stored in directory: /root/.cache/pip/wheels/02/62/46/93c713a5f061aeeb4f16eb6bf5ee798816e6ddda70faa78e69
Successfully built bert erlastic


### Data Preporcessing

In [5]:
#cols = ['sentiment', 'id', 'date', 'query', 'user', 'text']
#data = pd.read_csv('training.csv', header = None, names = cols, encoding = 'latin')
data = pd.read_csv('training.csv', header = None, encoding = 'latin')#
dta.head()

FileNotFoundError: ignored

In [0]:
data.drop(['id', 'date', 'query', 'user'], axis = 1, inplace = True)

KeyError: ignored

In [0]:
data.sentiment.value_counts()

4    800000
0    800000
Name: sentiment, dtype: int64

In [0]:
data.head()

,sentiment,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


Cleaning the tweets

In [0]:
def clean_tweets(tweet):
    tweet = BeautifulSoup(tweet, "lxml").get_text()
    tweet = re.sub(r"@[A-Za-z0-9]+", ' ', tweet)
    tweet = re.sub(r"https?://[A-Za-z0-9./]+", ' ', tweet)
    tweet = re.sub(r"[^a-zA-Z.!?']", ' ', tweet)
    tweet = re.sub(r" +", ' ', tweet)
    return tweet

In [0]:
data_clean = [clean_tweets(tweet) for tweet in data.text]

C:\Users\dobesz.Andreas-PC\anaconda3\envs\kerasplus\lib\site-packages\bs4\__init__.py:312: UserWarning: " i just received my G8 viola exam.. and its... well... .. disappointing.. :\.." looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % self._decode_markup(markup)
C:\Users\dobesz.Andreas-PC\anaconda3\envs\kerasplus\lib\site-packages\bs4\__init__.py:312: UserWarning: "E3 ON PLAYSTATION HOME IN ABOUT AN HOUR!!!!!!!!!! \../  \../" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % self._decode_markup(markup)


In [0]:
data_labels = data.sentiment.values
data_labels[data_labels == 4] = 1

In [0]:
data.sentiment.value_counts()

1    800000
0    800000
Name: sentiment, dtype: int64

### BERT Keras layer 

Creating a BERT layer will provide access to meta data for tokenizer (ust like vocab size)

We first create an object of the FullTokenizer class, and an BERT embedding layer from the BERT model from hub.KerasLayer. The trainable parameter is set to False, which means that we will not be training the BERT embedding. A BERT vocabulary file is created, as a numpy array. We then set the text to lowercase and finally we pass our vocabulary_file and to_lower_case variables to the BertTokenizer object.

In [0]:
bert_url = 'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1'
Tokenizer = bert.bert_tokenization.FullTokenizer
bert_layer = hub.KerasLayer(bert_url, trainable= False)

In [0]:
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = Tokenizer(vocab_file, lower_case)

### Sensecheck for BERT tokenizer

Map the token strings to their vocabulary indeces.

In [0]:
tokenizer.tokenize("Telekom is the best company to work ")

['tel', '##ek', '##om', 'is', 'the', 'best', 'company', 'to', 'work']

In [0]:
tokenizer.convert_tokens_to_ids(tokenizer.tokenize("Telekom is the best company to work "))

[10093, 5937, 5358, 2003, 1996, 2190, 2194, 2000, 2147]

### Tokenize the sample tweets data

In [0]:
def encode_sentence(sent):
    return ["[CLS]"] + tokenizer.tokenize(sent) + ["[SEP]"]

In [0]:
data_inputs = [encode_sentence(sent) for sent in data_clean]

### Data Creation

Bert requires 3 different input for each sentence
1. convert id to token
2. padding token/ masking
3. segment(0 indicate 1st sentence, 1 indicate token of second sentence.

In [0]:
def get_ids(token):
    return tokenizer.convert_tokens_to_ids(token)

def get_mask(token):
    # this will return 1 if padding is not required  and 0 is padding is require
    return np.char.not_equal(token, "[PAD]").astype(int) 

#identify if it is 1st sent or 2nd sentence
def get_segments(token):
    seg_ids=[]
    current_seg_ids=0
    for tok in token:
        seg_ids.append(current_seg_ids)
        if tok == '[SEP]':
            current_seg_ids = 1-current_seg_ids
    return seg_ids

In [0]:
data_len = [[sent, data_labels[i], len(sent)] for i, sent in enumerate(data_inputs)]

random.shuffle(data_len)
data_len.sort(key=lambda x: x[2])

#select only first 2 columns and where sentence_length is greater than 7
sorted_all = [([get_ids(sent_lab[0]),
               get_mask(sent_lab[0]),
               get_segments(sent_lab[0])], sent_lab[1]) for sent_lab in data_len if sent_lab[2] > 7]

### Sensecheck

This returns 2 tensors: 
    we use 1st tensor when we are doing any classification task, 
    and 2nd is used when we want token wise specification

In [0]:
my_sent = ["[CLS]"] + tokenizer.tokenize("Roses are red. Water is blue") + ["[SEP]"]
bert_layer([tf.expand_dims(tf.cast(get_ids(my_sent),tf.int32), 0), 
            tf.expand_dims(tf.cast(get_mask(my_sent),tf.int32), 0),
            tf.expand_dims(tf.cast(get_segments(my_sent),tf.int32), 0)])

[<tf.Tensor: shape=(1, 768), dtype=float32, numpy=
 array([[-0.8880781 , -0.49188244, -0.94932246,  0.8968192 ,  0.755771  ,
         -0.22770368,  0.8709753 ,  0.33833498, -0.7908284 , -0.9999901 ,
         -0.6133331 ,  0.9477908 ,  0.9889228 ,  0.53215766,  0.94513303,
         -0.55921215, -0.2919084 , -0.6994527 ,  0.47294235, -0.43830714,
          0.8081388 ,  0.9999961 , -0.303607  ,  0.28442976,  0.5896429 ,
          0.9904285 , -0.7335669 ,  0.9400157 ,  0.96096396,  0.78111243,
         -0.61328346,  0.28224257, -0.99593884, -0.19647846, -0.9765973 ,
         -0.9966745 ,  0.51982385, -0.6446738 ,  0.01374471,  0.06916006,
         -0.9207242 ,  0.4120631 ,  0.9999898 ,  0.22642383,  0.651158  ,
         -0.29540482, -1.        ,  0.5123887 , -0.8712981 ,  0.92095476,
          0.89726746,  0.89020276,  0.14599545,  0.55230975,  0.5531425 ,
         -0.3581166 , -0.04199617,  0.3269583 , -0.27745858, -0.6523695 ,
         -0.63723606,  0.54434645, -0.9406472 , -0.9209846 , 

### Generator

As all of our sentences are in unequal length we cannot build our dataset using tensor. For this we will use the generator

In [0]:
all_dataset = tf.data.Dataset.from_generator(lambda: sorted_all, output_types = (tf.int32, tf.int32))

# This code did not work - see next cell.
BATCH_SIZE = 32
all_batched = all_dataset.padded_batch(BATCH_SIZE, padded_shapes=(None,()), padding_values=(0, 0))

In [0]:
# Alternative version for parameters:
# padded_shapes= tf.TensorShape(None,()) or ((None, ), ()) 
# padding_values= (0, 0) or (tf.constant(0), tf.constant(0))
BATCH_SIZE = 32
all_batched = all_dataset.padded_batch(BATCH_SIZE,padded_shapes=((None, None),()))

In [0]:
len(list(all_batched))

45136

In [0]:
len(list(sorted_all))

1444341